In [ ]:
# -*- coding: utf-8 -*-
"""C-SECTION.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/18eU9J00O-PJrJAx4dliO80tMhjt-EVAP
"""

In [ ]:
import pickle
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import tensorflow as tf
import scikeras
import warnings
from tensorflow import get_logger
from scikeras.wrappers import KerasClassifier, KerasRegressor
import keras
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.models import Sequential

In [ ]:
"""Preprocessing"""

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter
df=pd.read_excel("Dataset.xlsx")
df.head()
df.columns=df.columns.str.strip().str.upper()

In [ ]:
df = df.drop(columns=['COUPLE SITUATION', 'COUNTRY OF ORIGIN', 'MATERNAL EDUCATION', 'DAILY ALCOHOL INTAKE', 'PARITY'])

In [ ]:
df['MEDICAL INDICATION'] = df['MEDICAL INDICATION'].str.strip().str.lower()
MEDICAL_INDICATION = {
    'agatamiento materno': 'maternal exhaustion',
    'expulsivo prolongado': 'prolonged second stage of labor',
    'patología materna': 'maternal pathology',
    'spbf': 'breech presentation'
}
df.replace({'MEDICAL INDICATION': MEDICAL_INDICATION}, inplace=True)

In [ ]:
df['EMERGENCY CESAREAN SECTION'] = df['EMERGENCY CESAREAN SECTION'].str.strip().str.lower()
EMERGENCY_CESAREAN_SECTION = {
    'group 3. parto estacionado': 'parked delivery',
    'group 4. desproporción pelvifetal': 'pelvifetal disproportion',
    'group 2. fracaso de inducción': 'induction failure',
    'group 5. miscelanea': 'mixed',
    'group 1. rpbf': 'rpbf'
}
df.replace({'EMERGENCY CESAREAN SECTION': EMERGENCY_CESAREAN_SECTION}, inplace=True)

In [ ]:
df['PROGRAMMED CESAREAN SECTION'] = df['PROGRAMMED CESAREAN SECTION'].str.strip().str.lower()
PROGRAMMED_CESAREAN_SECTION = {
    'placenta previa oclusiva': 'occlusive placenta previa',
    'presentación podalic': 'podalic presentation',
    'casos especiales': 'special cases',
    'cirugía uterina previa': 'previous uterine surgery',
    'presentación transversa': 'transverse presentation',
    'primer gemelo no cefálica': 'first non-cephalic twin',
    'riesgo fetal sin posibilidad de parto': 'fetal risk without possibility of delivery'
}
df.replace({'PROGRAMMED CESAREAN SECTION': PROGRAMMED_CESAREAN_SECTION}, inplace=True)

In [ ]:
df['ARTHERIAL EMBOLIZATION'] = df['ARTHERIAL EMBOLIZATION'].str.strip().str.lower()
ARTHERIAL_EMBOLIZATION  = {
  'masculino': 'male',
  'femenino': 'female'
}
df.replace({'ARTHERIAL EMBOLIZATION' : ARTHERIAL_EMBOLIZATION }, inplace=True)

In [ ]:
df['SEX FETUS 1'] = df['SEX FETUS 1'].str.strip().str.lower()
SEX_FETUS_1  = {
  'extrahospitalario': 'out of hospital'
}
df.replace({'SEX FETUS 1' : SEX_FETUS_1 }, inplace=True)

In [ ]:
df['WEIGHT FETUS 1'] = df['WEIGHT FETUS 1'].str.strip().str.lower()
WEIGHT_FETUS_1  = {
  'desconocido': 'a stranger'
}
df.replace({'WEIGHT FETUS 1' : WEIGHT_FETUS_1 }, inplace=True)

In [ ]:
df['EXITUSFETAL FETUS 1'] = df['EXITUSFETAL FETUS 1'].str.strip().str.lower()
EXITUSFETAL_FETUS_1  = {
  '0: no precisa': '0: not required',
  '1: aspiración nasofaríngea': '1: nasopharyngeal aspiration',
  '2: aspiración nasofaríngea con oxígeno': '2: nasopharyngeal aspiration with oxygen',
  '3: aspiración nasofaríngea con oxígeno y ambú': '3: nasopharyngeal aspiration with oxygen and ambu',
  '5: aspiración nasofaríngea con oxígeno, ambú, masaje cardíaco y fármacos': '5: nasopharyngeal aspiration with oxygen, ambu, cardiac massage and drugs'
}
df.replace({'EXITUSFETAL FETUS 1' : EXITUSFETAL_FETUS_1 }, inplace=True)

In [ ]:
df['TYPE OF BIRTH'] = df['TYPE OF BIRTH'].str.strip().str.lower()
#df['CLASSIFICATION OF BIRTH'] = df['TYPE OF BIRTH'].apply(lambda x: 'c-section' if x in ['emergency c-section', 'ces programmed'] else 'normal')

In [ ]:
#df[['TYPE OF BIRTH', 'CLASSIFICATION OF BIRTH']].head()
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.dropna(axis=1, how='all')
imputer = SimpleImputer(strategy='most_frequent')
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
# Values to remove
values_to_remove = ['esp', 'forc', 'nalgas vaginal']

In [ ]:
# Remove rows where 'TYPE OF BIRTH' column contains these values
df = df[~df['TYPE OF BIRTH'].isin(values_to_remove)]

In [ ]:
label_encoders = {}
for column in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [ ]:
X = df.drop(columns=['TYPE OF BIRTH'])
y = df['TYPE OF BIRTH']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
print(X_train.shape, y_train.shape)

In [ ]:
smote=SMOTE(random_state=42)
X_resampled,y_resampled=smote.fit_resample(X_train,y_train)
y_train=y_resampled
print("After resampling:",Counter(y_train))
scaler = StandardScaler()
X_train = scaler.fit_transform(X_resampled)
X_test = scaler.transform(X_test)

In [ ]:
"""K-Nearest Neighbors"""

In [ ]:
print("KNN MODEL RESULTS")

In [ ]:
knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [ ]:
print("\nConfusion Matrix for KNN:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report for KNN:")
print(classification_report(y_test, y_pred))

In [ ]:
KNN_accuracy = accuracy_score(y_test, y_pred)*100
KNN_precision = precision_score(y_test,y_pred,average="macro")*100
KNN_recall = recall_score(y_test,y_pred,average="macro")*100
KNN_f1score = f1_score(y_test,y_pred,average="macro")*100
print(f"Accuracy of KNN: {KNN_accuracy:.2f}%")

In [ ]:
"""Logistic Regression"""

In [ ]:
X_resampled,y_resampled=smote.fit_resample(X,y)
print("LOGISTIC REGRESSION MODEL RESULTS")
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

In [ ]:
y_pred_logreg = logreg.predict(X_test)

In [ ]:
print("\nConfusion Matrix for Logistic Regression:")
print(confusion_matrix(y_test, y_pred_logreg))
print("\nClassification Report for Logistic Regression:")
print(classification_report(y_test, y_pred_logreg))

In [ ]:
logreg_accuracy = accuracy_score(y_test, y_pred_logreg)*100
logreg_precision = precision_score(y_test,y_pred_logreg,average="macro")*100
logreg_recall = recall_score(y_test,y_pred_logreg,average="macro")*100
logreg_f1score = f1_score(y_test,y_pred_logreg,average="macro")*100
print(f"Accuracy of Logistic Regression: {logreg_accuracy:.2f}%")

In [ ]:
"""Decision Tree"""

In [ ]:
print("DECISION TREE MODEL RESULTS")
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

In [ ]:
y_pred_dt = dt.predict(X_test)

In [ ]:
print("\nConfusion Matrix for Decision Tree Classifier:")
print(confusion_matrix(y_test, y_pred_dt))
print("\nClassification Report for Decision Tree Classifier:")
print(classification_report(y_test, y_pred_dt))

In [ ]:
dectree_accuracy = accuracy_score(y_test, y_pred_dt)*100
dectree_precision = precision_score(y_test,y_pred_dt,average="macro")*100
dectree_recall = recall_score(y_test,y_pred_dt,average="macro")*100
dectree_f1score = f1_score(y_test,y_pred_dt,average="macro")*100
print(f"Accuracy of Decision Tree Classifier: {dectree_accuracy:.2f}%")

In [ ]:
"""Neural Networks"""

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
def create_model():
    model = Sequential()
    model.add(Input(shape=(49,)))
    model.add(Dense(6, kernel_initializer=tf.keras.initializers.GlorotNormal(seed=42), activation='relu'))
    model.add(Dense(4, kernel_initializer=tf.keras.initializers.GlorotNormal(seed=42), activation='relu'))
    model.add(Dense(1, kernel_initializer=tf.keras.initializers.GlorotNormal(seed=42), activation='sigmoid'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
print("Neural network processing...")

In [ ]:
estimator = KerasClassifier(model=create_model, epochs=30, batch_size=32, verbose=1, random_state=42)

In [ ]:
# After training the neural network
estimator.fit(X_train, y_train)

In [ ]:
neural_pred = estimator.predict(X_test)
nn_accuracy = accuracy_score(y_test, neural_pred)*100
nn_precision = precision_score(y_test,neural_pred,average="macro")*100
nn_recall = recall_score(y_test,neural_pred,average="macro")*100
nn_f1score = f1_score(y_test,neural_pred,average="macro")*100
print(f"Accuracy of neural network: {nn_accuracy:.2f}%")

In [ ]:
# Example metrics for different models
results = {
    "K Nearest Neighbours": {
        "accuracy": KNN_accuracy,
        "precision": KNN_precision,
        "recall": KNN_recall,
        "f1_score": KNN_f1score
    },
    "Decision Tree": {
        "accuracy": dectree_accuracy,
        "precision": dectree_precision,
        "recall": dectree_recall,
        "f1_score": dectree_f1score
    },
    "Logistic Regression": {
        "accuracy": logreg_accuracy,
        "precision": logreg_precision,
        "recall": logreg_recall,
        "f1_score": logreg_f1score
    },
    "Neural Network":{
      "accuracy": nn_accuracy,
        "precision": nn_precision,
        "recall": nn_recall,
        "f1_score": nn_f1score
    }
}

In [ ]:
# Save to JSON file
import json
with open("results_2.json", "w") as f:
    json.dump(results, f)